# Analysis of LLM Output

This notebook contains the analysis for the output of the LLM data extraction process contained in the CCI Literature Analysis with LLM notebook. It processes the output dataframe to standardize the responses, then compiles and analyzes the references across a few different axes. 

Author: Josh Fuchs

Copyright 2025, The University of North Carolina at Chapel Hill. Permission is granted to use in accordance with the MIT license. The code is licensed under the open-source MIT license.


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from typing import Union

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

import seaborn as sns

%matplotlib inline

## Load Output DataFrame

In [ ]:
result_df = pd.read_csv('YOUR_PATH_HERE',index_col=0)

## DataFrame Preprocessing for Analysis

Because the responses from the LLM are not completely standardized, there are three pre-analysis steps we need to take with the dataframe before conducting the final analysis. 

1. Standardize formatting for the columns of interest.
2. Process papers that have multiple paragraphs. 
3. Standardize the response references.

### 1. Column Formatting

In [ ]:
# Columns to format
columns_to_format = ['q0', 'q1', 'q2']

# Apply formatting: lowercase and remove periods
for col in columns_to_format:
    result_df[col] = result_df[col].str.lower().str.replace('.', '', regex=False)

# Columns Q3 and Q4 have the results of the data extraction, so
# they require some extra formatting to ensure consistency

# Remove 'et al.,' from q3 and q4 columns
result_df['q3'] = result_df['q3'].str.replace('et al.,', '', regex=False)
result_df['q4'] = result_df['q4'].str.replace('et al.,', '', regex=False)

# Replace all double spaces with single space
result_df['q3'] = result_df['q3'].str.replace('  ', ' ', regex=False)
result_df['q4'] = result_df['q4'].str.replace('  ', ' ', regex=False)

# Replace None with NONE
result_df['q3'] = result_df['q3'].str.replace('None', 'NONE', regex=False)
result_df['q4'] = result_df['q4'].str.replace('None', 'NONE', regex=False)

# Format column data types
result_df['pmc'] = result_df['pmc'].astype('int')
result_df['tokens_used'] = result_df['tokens_used'].astype('float')

### 2. Deal with papers that have mutliple paragraphs analyzed

Some articles have multiple paragraphs that contain references we are interested in. So these PMC IDs appear multiple times in our dataframe. Here we will de-duplicate these duplicated PMC IDs. We will also combine the extracted references to a single column named 'ref' so we can operate on a single column going forward. 

We will take the following steps to do this: 

1. Separate the duplicated and non-duplicated PMCs. Add the number of paragraphs present for all. 
2. For the PMCs that are duplicated, apply the following logic to determine the set of references used. This is implemented in the handle_duplicates function:
    1. If the extracted reference is the same for all paragraphs, that is the reference. Keep the first occurrence.
    2. If different paragraphs have different numbers of references, but they overlap (i.e. Charlson 1987, Quan 2005 then Charlson 1987), select the paragraph that has the most references. 
    3. If paragraphs have all NONE except one row, then keep the one row that is not NONE.
    4. If paragraphs have separate references (i.e. Charlson 1987 then Quan 2005), combine all references into a single set. 
3. Concatenate duplicated and non-duplicated dataframes back together. 

In [ ]:
def determine_ref(row: pd.Series) -> Union[str,float]:
    '''
    Combines references, which were determined in either Q3 or Q4,
    into a single column. Assumes NaN response when that question
    was not used

    PARAMETERS:
        row: a row of the dataframe

    RETURNS:
        a string containing the references
    '''
    if pd.isna(row['q3']) and pd.isna(row['q4']):
        return np.nan
    elif pd.isna(row['q3']):
        return row['q4']
    elif pd.isna(row['q4']):
        return row['q3']
    else:
        return 'warning'

In [ ]:
# Merge the extracted references to a single
# column named 'ref' using the determine_ref function
result_df['ref'] = result_df.apply(determine_ref, axis=1)

# Identify duplicated rows based on the 'PMC' column
duplicated_pmc = result_df[result_df.duplicated('pmc', keep=False)].copy()

# Add in a column that counts how many times each PMC value is duplicated
duplicated_pmc.loc[:, 'paragraphs'] = duplicated_pmc.groupby('pmc')['pmc'].transform('count')

# Drop the duplicated PMC rows from the original dataframe
# This creates a dataframe where each PMC only appears once
# We'll merge these together again later
result_single_pmc_df = result_df.drop(duplicated_pmc.index)

# Add in a column to keep track of number of extracted paragraphs
# for the single PMC dataframe, this will be 1 unless 
result_single_pmc_df['paragraphs'] = np.where(result_single_pmc_df['paragraph'].isna(), 0, 1)


In [ ]:
# Functions to handle each group of duplicates

def process_refs(refs: list) -> str:
    """
    Processes a list of reference strings, splitting them into sets and determining if any set is a superset of the others.

    PARAMETERS:
    refs : list
        A list of reference strings, where each string contains references separated by commas.

    RETURNS:
    str : A sorted, comma-separated string of references. If any set is a superset of all other sets, returns the sorted superset. Otherwise, returns a sorted combination of all references.
    """

    # Split references into sets, only if they are strings
    ref_sets = [set(ref.split(', ')) for ref in refs if isinstance(ref, str)]
    
    # Check if any set is a superset of others
    for i in range(len(ref_sets)):
        if all(ref_sets[i].issuperset(ref_sets[j]) for j in range(len(ref_sets)) if i != j):
            return ', '.join(sorted(ref_sets[i]))
    
    # Combine all references into a single set
    combined_refs = set().union(*ref_sets)
    return ', '.join(sorted(combined_refs))

def handle_duplicates(group: pd.DataFrame) -> pd.DataFrame:
    """
    Handles duplicate references within a group of paragraphs by 
    determining the appropriate row to keep based on the values of the 'ref' column.

    PARAMETERS:
    group : DataFrame
        A pandas DataFrame representing a group of rows with a 'ref' column containing reference values.

    RETURNS:
    DataFrame: A DataFrame containing only the row to keep based on the following conditions:
        - If all 'ref' values are the same, keeps the first row.
        - If all 'ref' values are 'NA' except for one, keeps the row with the non-'NA' value.
        - Otherwise, processes the references using the `process_refs` function and updates the first row with the summarized reference, keeping only that row.
    """

    unique_ref_values = group['ref'].unique()
    
    if len(unique_ref_values) == 1:
        # If the value of 'ref' is the same for all, keep only the first row of that pmc group
        return group.iloc[[0]]
    elif pd.isna(group['ref']).sum() == len(group) - 1:
        # If the value of 'ref' is 'NA' for all except 1 row, keep only that row of the pmc group
        return group[~pd.isna(group['ref'])].iloc[[0]]
    else:
        # else, process refs using the process_refs function
        # basically, we'll combine all the references into a single set
        refs = group['ref'].tolist()
        summarized_ref = process_refs(refs)
        
        # Update the first row with the summarized ref and keep only that row
        group.iloc[0, group.columns.get_loc('ref')] = summarized_ref
        return group.iloc[[0]]

In [ ]:
# Apply the function to each group of duplicates and concatenate the results
duplicated_pmc_reduced = duplicated_pmc.groupby('pmc').apply(handle_duplicates).reset_index(drop=True)

In [ ]:
# Combine the duplicated and non-duplicated dataframes together
cleaned_result_df = pd.concat([result_single_pmc_df, duplicated_pmc_reduced], axis=0,ignore_index=True)

# Replace NaN values in the ref column with 'NOREF'
# None means there was a reference but it wasn't used
# NaN means there was no reference
cleaned_result_df['ref'] = cleaned_result_df['ref'].fillna('NOREF')
cleaned_result_df['ref'] = cleaned_result_df['ref'].astype(str)

### 3. Standardize the responded references

Remove any references that are not to the selected CCI versions and ensure that references are formatted the same, with a comma between each for automated extraction. 


In [ ]:
# Replace 'and' with ',' in 'ref' column and save to 'ref_clean' column
cleaned_result_df['ref_clean'] = cleaned_result_df['ref'].str.replace(r'\s+and\s+', ',', regex=True,case=False)

# Remove any instances of et al. or et al 
# We do the replacement directly on the ref_clean column so that we do not overwrite
# the previous work
cleaned_result_df['ref_clean'] = cleaned_result_df['ref_clean'].str.replace('et al.', '',regex=True)
cleaned_result_df['ref_clean'] = cleaned_result_df['ref_clean'].str.replace('et al', '',regex=True)
cleaned_result_df['ref_clean'] = cleaned_result_df['ref_clean'].str.replace('et al,', '',regex=True)

# Replace any ; with ,
cleaned_result_df['ref_clean'] = cleaned_result_df['ref_clean'].str.replace(';', ',',regex=True)

# Replace any double spaces '  ' with single space ' '
cleaned_result_df['ref_clean'] = cleaned_result_df['ref_clean'].str.replace('  ', ' ',regex=True)

In [ ]:
def filter_references(ref_list: str,valid_references: str) -> str:
    """
    Filters a list of references, ensuring each reference appears only once and is valid.
    Removes 'NONE' and 'NOREF' if there are other valid references.
    Adds 'OTHER' if there are references not in valid_references.

    PARAMETERS:
    ref_list : str
        A comma-separated string of references.
    valid_references : set
        A set of valid references.

    RETURNS:
    str: A comma-separated string of filtered references. If no valid references are found, returns "OTHER".
    """
    refs = ref_list.split(',')
    # Use set to ensure each reference only appears once
    filtered_refs = {ref.strip() for ref in refs if ref.strip() in valid_references}
    
    # Check for invalid references and add 'OTHER' if any are found
    if any(ref.strip() not in valid_references for ref in refs):
        filtered_refs.add('OTHER')
    
    # Remove 'NONE' and 'NOREF' if there are other valid references
    if filtered_refs - {'NONE', 'NOREF'}:
        filtered_refs.discard('NONE')
        filtered_refs.discard('NOREF')
    
    return ', '.join(filtered_refs) if filtered_refs else "OTHER"

In [ ]:
# List of valid references
# Keep NOREF and NONE because they indicate something
# If the reference is not in the list, we'll return OTHER
valid_references = ['Charlson 1987', 'Deyo 1992', 'Quan 2005', 'Quan 2011', 'Klabunde 2000',
                    'Sundararajan 2004', 'Schneeweiss 2003','Halfon 2002', 'Charlson 1994',
                    'Romano 1993', 'NOREF','NONE']

# Apply the function to the ref_clean column
cleaned_result_df['ref_clean_filtered'] = cleaned_result_df['ref_clean'].apply(lambda x: filter_references(x,valid_references))

## Final Dataframe Analysis

Here are the questions we are interested in answering:
1. How many papers cite none of the major versions?
2. How many papers reference a CCI version, but don't actually calculate it? 
3. How often is each CCI version cited as a single reference?
4. How often is each CCI version used, either individually or in combination? 
6. How often is there more than one paper cited, but our model picks out fewer as the versions used? 

In [ ]:
print("The number of papers analyzed is {}".format(cleaned_result_df.shape[0]))

In [ ]:
print("The number of papers with one paragraph extracted is {}".format(cleaned_result_df[cleaned_result_df['paragraphs'] == 1].shape[0]))

In [ ]:
print("The number of papers with more than one paragraph extracted is {}".format(cleaned_result_df[cleaned_result_df['paragraphs'] > 1].shape[0]))

In [ ]:
print("The number of papers with no paragraphs extracted is {}".format(cleaned_result_df[cleaned_result_df['paragraphs'] < 1].shape[0]))

### 1. How many papers cite none of the CCI versions? 

To calculate this we want to select where included_references == NaN

In [ ]:
no_citations = cleaned_result_df[cleaned_result_df['included_references'].isna()].shape[0]
print("The number of papers with no citations is {}".format(no_citations))

### 2. How many papers reference a Charlson version, but don't actually calculate it? 

This can be answered with either Q0 = no, Q1 = no, or ref_clean_filtered = None.

In [ ]:
no_calculation = cleaned_result_df[(cleaned_result_df['q0'] == "no") 
                                   | (cleaned_result_df['q1'] == "no")
                                   | (cleaned_result_df['ref_clean_filtered'] == "NONE")]
print("The number of papers with no calculation is {}".format(no_calculation.shape[0]))

### 3. How often is each CCI version cited as a single reference?


In [ ]:
versions = ['Charlson 1987', 'Deyo 1992', 'Romano 1993','Charlson 1994',
            'Klabunde 2000','Halfon 2002','Schneeweiss 2003','Sundararajan 2004', 
            'Quan 2005', 'Quan 2011']

for x in versions:
    version_count = cleaned_result_df[cleaned_result_df['ref_clean_filtered'] == x].shape[0]
    print("The number of papers that only cite {} is {}".format(x, version_count))

### 4. How often is each CCI version used, either individually or in combination?

Here we will count versions that are part of multiple reference groups, such as citing both Charlson 1987 and Quan 2005 in the same paper. 

In [ ]:
def count_unique_references(df: pd.DataFrame, column_name: str) -> Counter[str]:    
    """
    Count the frequency of unique references in a specified column of a DataFrame.

    This function processes each entry in the given column, assuming entries contain
    comma-separated references. It strips whitespace and common punctuation from each
    reference, then counts the occurrences of each unique reference across all entries.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        column_name (str): The name of the column to analyze.

    Returns:
        collections.Counter: A Counter object mapping each unique reference to its frequency.
    """

    # Initialize a Counter to store the counts of unique references
    reference_counter = Counter()
    
    # Iterate over each entry in the specified column
    for entry in df[column_name]:
        # Convert the entry to a string and split by comma
        references = [ref.strip().strip("',[]") for ref in str(entry).split(',')]
        # Update the counter with the references
        reference_counter.update(references)
    
    return reference_counter

In [ ]:
# Count unique references
unique_refs = count_unique_references(cleaned_result_df,'ref_clean_filtered')
print(unique_refs)

### 5. How often is there more than one paper cited, but our model picks out fewer as the versions used?


In [ ]:
# Exclude NaNs from included_references and calculate num_included_references as 1 + number of commas
cleaned_result_df['num_included_references'] = cleaned_result_df['included_references'].apply(lambda x: 1 + x.count(',') if isinstance(x, str) else 0)

# Calculate num_ref_clean_filtered as 1 + number of commas in ref_clean_filtered
# if the value is None or 'NOREF', set num_ref_clean_filtered to 0
cleaned_result_df['num_ref_clean_filtered'] = cleaned_result_df['ref_clean_filtered'].apply(
    lambda x: 0 if x in [None, 'NOREF'] else 1 + x.count(',')
    )

# Compare num_included_references and num_ref_clean_filtered
comparison_count = (cleaned_result_df['num_included_references'] > cleaned_result_df['num_ref_clean_filtered']).sum()

print(f"There are {cleaned_result_df[cleaned_result_df['num_included_references'] > 1].shape[0]} papers that cite more than one version of CCI.")
print(f"There are {cleaned_result_df[cleaned_result_df['num_ref_clean_filtered'] > 1].shape[0]} papers that use more than version of CCI.")
print(f"The number of times num_included_references is larger than num_ref_clean_filtered is {comparison_count}. This is different than above because this includes times when there would be a single version used. ")


## Analysis By Year

In [ ]:
# Identify single citations to the different versions

version_count = cleaned_result_df[cleaned_result_df['ref_clean_filtered'] == 'Charlson 1987']
version_count.groupby("pub_year")

In [ ]:
# Identify any reference to the versions, including single and multiples
filtered_df = cleaned_result_df[cleaned_result_df['ref_clean_filtered'].str.contains('Charlson 1994', na=False)]

# Group by the pub_year column and count the occurrences
grouped_counts = filtered_df.groupby('pub_year').size()

# Print the counts per pub_year
print(grouped_counts)

### Plot Percentages over year

In [ ]:
# Removed Halfon 2002 since it has zero single references
versions_limited = ['Charlson 1987', 'Deyo 1992', 'Romano 1993',
            'Charlson 1994', 'Klabunde 2000', 'Schneeweiss 2003', 
            'Sundararajan 2004', 'Quan 2005', 'Quan 2011'
            ]

# Select rows that exactly match any of the strings in the versions
filtered_df = cleaned_result_df[cleaned_result_df['ref_clean_filtered'].isin(versions_limited)]

# Group by the pub_year column and count the occurrences for each string
grouped_counts = filtered_df.groupby(['pub_year', 'ref_clean_filtered']).size().unstack(fill_value=0)

# Calculate the total counts per year
total_counts_per_year = grouped_counts.sum(axis=1)#filtered_df.groupby('pub_year').size()

# Calculate the percentage for each string per year
percentages = grouped_counts.div(total_counts_per_year, axis=0) * 100

# Reset index for seaborn plotting
percentages = percentages.reset_index()

# Melt the DataFrame for seaborn plotting
percentages_melted = percentages.melt(id_vars='pub_year', value_vars=versions_limited, var_name='String', value_name='Percentage')

In [ ]:
# Get unique strings
unique_strings = percentages_melted['String'].unique()

# Create a 3x3 grid of plots
fig, axes = plt.subplots(3, 3, figsize=(12, 5), sharex=True)

# Flatten the axes array for easy iteration
axes = axes.flatten()

color_dict = {'Charlson 1987' : 'red',
              'Deyo 1992' : 'red',
              'Romano 1993' : 'black',
            'Charlson 1994' : 'green',
            'Klabunde 2000' : 'black',
            'Schneeweiss 2003' : 'black', 
            'Sundararajan 2004' : 'black',
            'Quan 2005' : 'green',
            'Quan 2011' : 'green'
}

# Iterate over unique strings and create a plot for each
for i, string in enumerate(unique_strings):
    sns.lineplot(
        data=percentages_melted[percentages_melted['String'] == string],
        x='pub_year',
        y='Percentage',
        ax=axes[i],
        color=color_dict[string],
        linewidth=2.5
    )
    axes[i].text(0.5,0.85,string,
                 fontsize=14,
                 horizontalalignment='center',
                 transform=axes[i].transAxes,)
    axes[i].set_xlabel('Publication Year', fontsize=14)
    axes[i].tick_params(axis='both', which='major', labelsize=14)
    axes[i].xaxis.set_minor_locator(MultipleLocator(1))
    if string in ('Charlson 1987', 'Charlson 1994', 'Sundararajan 2004'):
        axes[i].set_ylabel('Percentage', fontsize=14)
    else:
        axes[i].set_ylabel('')
    if string == 'Charlson 1987':
        axes[i].set_ylim(0,100)
        axes[i].yaxis.set_minor_locator(MultipleLocator(25))
    else:
        axes[i].set_ylim(0,20)
        axes[i].yaxis.set_minor_locator(MultipleLocator(5))
plt.show()